## Deliverable 2. Create a Customer Travel Destinations Map.

In [75]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [38]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv('/Users/kevindaniliuk/Desktop/Modules/Module6/world_weather_analysis/Weather_Database/WeatherPy_Database.csv')
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Carauari,BR,-4.8828,-66.8958,86.18,52,80,2.08,broken clouds
1,1,Norton,ZW,-17.8833,30.7000,56.28,60,0,8.84,clear sky
2,2,Gizo,SB,-8.1030,156.8419,82.26,75,38,6.60,scattered clouds
3,3,Busselton,AU,-33.6500,115.3333,47.44,84,83,5.46,broken clouds
4,4,Yar-Sale,RU,66.8333,70.8333,31.87,83,75,8.46,broken clouds


In [39]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [41]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Carauari,BR,-4.8828,-66.8958,86.18,52,80,2.08,broken clouds
2,2,Gizo,SB,-8.1030,156.8419,82.26,75,38,6.60,scattered clouds
13,13,Shingu,JP,33.7333,135.9833,77.11,88,98,5.99,overcast clouds
20,20,Baijiantan,CN,45.6333,85.1833,77.68,16,0,2.21,clear sky
23,23,Orikum,AL,40.3253,19.4714,75.70,70,0,5.57,clear sky
24,24,Alexandria,EG,31.2156,29.9553,78.03,75,14,8.75,few clouds
25,25,Sinnamary,GF,5.3833,-52.9500,80.17,81,73,11.16,broken clouds
27,27,Vaini,TO,-21.2000,-175.2000,80.76,89,20,8.05,few clouds
31,31,Jega,NG,12.2175,4.3792,75.36,90,100,5.79,overcast clouds
32,32,Ribeira Grande,PT,38.5167,-28.7000,75.99,91,100,18.30,light rain


In [55]:
# 4a. Determine if there are any empty rows.
print(len(preferred_cities_df))
preferred_cities_df.notnull().sum()

239


Unnamed: 0             239
City                   239
Country                239
Lat                    239
Lng                    239
Max Temp               239
Humidity               239
Cloudiness             239
Wind Speed             239
Current Description    239
dtype: int64

In [52]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# no empty rows. 

#if there were empty rows:
clean_preferred_cities_df = preferred_cities_df.dropna(axis="index", how="any")
clean_preferred_cities_df.count()

Unnamed: 0             239
City                   239
Country                239
Lat                    239
Lng                    239
Max Temp               239
Humidity               239
Cloudiness             239
Wind Speed             239
Current Description    239
dtype: int64

In [56]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Carauari,BR,86.18,broken clouds,-4.8828,-66.8958,
2,Gizo,SB,82.26,scattered clouds,-8.1030,156.8419,
13,Shingu,JP,77.11,overcast clouds,33.7333,135.9833,
20,Baijiantan,CN,77.68,clear sky,45.6333,85.1833,
23,Orikum,AL,75.70,clear sky,40.3253,19.4714,
24,Alexandria,EG,78.03,few clouds,31.2156,29.9553,
25,Sinnamary,GF,80.17,broken clouds,5.3833,-52.9500,
27,Vaini,TO,80.76,few clouds,-21.2000,-175.2000,
31,Jega,NG,75.36,overcast clouds,12.2175,4.3792,
32,Ribeira Grande,PT,75.99,light rain,38.5167,-28.7000,


In [78]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [86]:
# 7. Drop the rows where there is no Hotel Name.

#7a. replace empty values with NaN.
hotel_df_replaced = hotel_df.replace("", np.nan, regex=True)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Carauari,BR,86.18,broken clouds,-4.8828,-66.8958,Hotel Fellip
2,Gizo,SB,82.26,scattered clouds,-8.1030,156.8419,Imagination Island
13,Shingu,JP,77.11,overcast clouds,33.7333,135.9833,Shingu UI Hotel
20,Baijiantan,CN,77.68,clear sky,45.6333,85.1833,NaN
23,Orikum,AL,75.70,clear sky,40.3253,19.4714,Hotel Denta
...,...,...,...,...,...,...,...
692,Praya,CV,81.14,clear sky,14.9215,-23.5087,LT Aparthotel
693,Mana,GF,79.16,few clouds,5.6592,-53.7767,LE SAMANA
697,Gushikawa,JP,81.63,clear sky,26.3544,127.8686,Kirari Guest House
698,Alim,PH,81.93,overcast clouds,9.5649,122.4876,Eden Island Resort and Spa


In [87]:
#7b. now drop the rows with NaN
cleaned_hotel_df = hotel_df_replaced.dropna(axis="index", how="any")
cleaned_hotel_df.head(11)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Carauari,BR,86.18,broken clouds,-4.8828,-66.8958,Hotel Fellip
2,Gizo,SB,82.26,scattered clouds,-8.1030,156.8419,Imagination Island
13,Shingu,JP,77.11,overcast clouds,33.7333,135.9833,Shingu UI Hotel
23,Orikum,AL,75.70,clear sky,40.3253,19.4714,Hotel Denta
24,Alexandria,EG,78.03,few clouds,31.2156,29.9553,Royal Crown Hotel
25,Sinnamary,GF,80.17,broken clouds,5.3833,-52.9500,ALETHEIA Appartement Sinnamary
27,Vaini,TO,80.76,few clouds,-21.2000,-175.2000,Keleti Beach Resort
31,Jega,NG,75.36,overcast clouds,12.2175,4.3792,ALH. Nasiru Ibrahim
32,Ribeira Grande,PT,75.99,light rain,38.5167,-28.7000,Quinta da Meia Eira
37,Kapaa,US,84.09,broken clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort


In [90]:
# 8a. Create the output File (CSV)
output_data_file = '/Users/kevindaniliuk/Desktop/Modules/Module6/world_weather_analysis/Vacation_Search/WeatherPy_vacation.csv'

# 8b. Export the City_Data into a csv
cleaned_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [112]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in cleaned_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = cleaned_hotel_df[["Lat", "Lng"]]

In [114]:
# 11a. Add a marker layer for each city to the map. 
locations = cleaned_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))